# IBM DATA SCIENCE CAPSTONE PROJECT

> Hello this the notebook for the capstone that involves locational data. Really missing ArcGIS from ESRI here but it should go well anyways 

In [ ]:
print("Hello Capstone Project Course")

# Introduction

### Failed Ideas

1. My hometown has  alot of grocery stores clustered in the middle of it and everyone as the city has grown has to drive in. I would like to identify where new grocery stores would be best put to reach a good number of people and be away form the others. That is to say to stop making people drive in so far to get a to a store.
2. Do the same as one but for gas stations which are in a similar situation.

#### Why They Failed 

Why they failed is due to the fact that my homeotwn of about 60,000 people being the second largest in my state is too small to have enough data to do this on. I can get the gas stations and grocery store locations. but the smallest geographical boundaries I could obtain where zipcodes. The zipcodes as shown on the official USA zipcode map had one small inner city one and then there was a second one that covered most of the rest the city. Then the edges of the city barely reached into 4 additioonal zipcode areas. These would not have been good break downs for a distribution within my city for the above problems. So I couldn't do anything smaller than a city level analysis in my hometown. 

#### Other Failed Ideas
I also considred that since my state has lots of National Forests and campgrounds. I could identify different qualities of campgrounds and build a state camping map by campsite quality. This failed as Foursquare does not contain any information for campgrounds in National Forest Land. While I could get that data from the National Forest Service directly, there would be no use of Foursquare in the analysis which is required.

#### So What did I do

I decided at this point that my hometown which I love is a bad idea for data science application. So, Idecided to go with a simpler and in the end a more personal application as follows below.

## The problem

#### The Audience

> My Audience is my Family and anyone else who wants to live in a quiter, less populous area, without all the fancy attractions but still has stuff to do simialr tomy Hometown of Idaho Falls, Idaho

#### The Background

> Recent college graduates and those just starting their own family often worry greatly about where to live. This is usually decided for them by a job as to what city, state or country thier job is loccated in. However within a given city, state or country, there is a great diversity between states, counties, or even different neighborhoods. Thus keepign in trying to live somewhere similar to where they grew up but still having opportunity to have work with good wages.

> Directly with my comparison my city I grew up in is small enough that many would laugh at it being called a city at around 60,000 people and about 5 square miles. Jobs are more limited to a few big employers or owning/managing your own business. Thus I adn many other young adults are lookign at moving to bigger cities even if that is not ideal for ourlifestyle, habits, and upbringing.

#### The Question

> Are there other Counties similar to the one I grew up otehr places in the Nation where job opportunites and wages are better?

#### Limitations

> As there are several thousand cities in America with great differences in size. I have decided instead of using cities to use counties which are a more equal distribution as large cities and metropolian areas that cover multiple counties can be split into pieces and small places where several cities are in a county can be grouped together. 

> Also, I am going to further restrict the counties to ones that are more simialr in population as well to further balance the scale appropriately. I will work with counties between 50,000 and 500,000 thousand people. This removes a little more than the top 100 most populous counties in the US acorrding to 2010 Census population counts as reported on teh following wikipedia page [Most Populous Counties in the US](https://en.wikipedia.org/wiki/List_of_the_most_populous_counties_in_the_United_States)

> I am basing similarity of counties on venue types as I believe that reflects the culture and wealth of the area quite well. That alongside the population should allow me to find areas similar to where I grew up.

# Data

### Outside of Foursquare

> I will use the county name and population columns of the table of American county information as from wikipedia's list here [Wikiepdia American Counties](https://en.wikipedia.org/wiki/List_of_United_States_counties_and_county_equivalents)

> County names lets me group my data geographically. County Population helsp to define simialrity between counties

### Within Foursquare
> I also will use Foursquare data of venue types as was done with the Toronto data as a measure of similarity of culture and wealth of the county.

# Methodology

### Step 1- Import libraries, functions and data

In [146]:
# libraries and functions
import pandas as pd 
import numpy as np
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print(pd.__version__) # Check that pandas 1.0.0 or newer is used

1.0.5


#### Note 
> Used trial and error to find index of 3 for the actual table from the webpage for above code. 
> for below code I did some trial and error and read through the foursquare api documentation and discovered that using near and not defining a raidus causes foursquare to base radius on density of veues of teh area given in near. That way I am getting teh venues returned form the most venue dense areas of each county regardless of size. Also the foursquare APi automatically geocodes which is really useful to save on coding. 

In [165]:
# foursquare data

def getNearbyVenues(names):
    LIMIT = 25
    CLIENT_ID = '0RGBNDWZRDACEKIXYD4TLZ24JCK2CHZ51TLYCDCPXYVRLDJP' # your Foursquare ID
    CLIENT_SECRET = '01B5TCEM3IUMXB1NKTQK4ACOKFEI525X2OLAW1WLUMCN0F1J' # your Foursquare Secret
    VERSION = '20200730' # Foursquare API version
    venues_list=[]
    for name in names:
        print(name)
        NEAR=name
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            NEAR, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print("failed to obtain information for " + name)
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['County',
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [153]:
# county data
america_counties=pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_counties_and_county_equivalents')[3]
america_counties

,County or equivalent,State or equivalent,Population (2019 estimate),Core Based Statistical Area
0,Autauga,Alabama,55869,"Montgomery, AL Metropolitan Statistical Area"
1,Baldwin,Alabama,223234,"Daphne-Fairhope-Foley, AL Metropolitan Statist..."
2,Barbour,Alabama,24686,NaN
3,Bibb,Alabama,22394,"Birmingham-Hoover, AL Metropolitan Statistical..."
4,Blount,Alabama,57826,"Birmingham-Hoover, AL Metropolitan Statistical..."
...,...,...,...,...
3240,Sweetwater,Wyoming,42343,"Rock Springs, WY Micropolitan Statistical Area"
3241,Teton,Wyoming,23464,"Jackson, WY-ID Micropolitan Statistical Area"
3242,Uinta,Wyoming,20226,"Evanston, WY Micropolitan Statistical Area"
3243,Washakie,Wyoming,7805,NaN


### Step 2 Wrangle the Data

> This involves cleaning, formating, and initial exploratory analysis. Most of the initial exploratory analysis was doen in the researching of the topic just by looking through the data nad thinking about the problem so only a few things will be noted.

In [154]:
#check dtypes
america_counties.dtypes

# convert dtypes. using pandas 1.0+ there is now a string type separate form object
america_countes=america_counties[['County or equivalent', 'State or equivalent']].astype("string")
#drop/delete unneeded column of Core based statistical area using python general delete command
del america_counties[america_counties.keys()[3]]
america_counties=america_counties.drop(america_counties[america_counties['County or equivalent'] == 'Bonneville'].index) #remove home county from list

In [159]:
#remove leftover non string items from web table which in this case where floating brackets for footnotes
temp_list=america_counties['County or equivalent'].str.split(pat='[', expand=True)
america_counties['County or equivalent']=temp_list[0]

#### Note
> other possible errors will be caught in the try exept block in the getNearbyVenues function and will get skipped

In [160]:
smaller_list=america_counties['Population (2019 estimate)'].between(50000, 500000).dropna()
america_counties[smaller_list].shape

(869, 3)

#### Note
> Due to needing venues per county and the limit on free api calls to the foursquare api I will use only a sample of 150 counties. Technically I could just barely make it under the rate limit for the full set of 870 but I will avoid that here while doing the project and save that for when everything is working

In [161]:
sample_list=america_counties[smaller_list].sample(n=150)
sample_list

,County or equivalent,State or equivalent,Population (2019 estimate)
1225,Berkshire,Massachusetts,124944
335,Clay,Florida,219252
1376,Otter Tail,Minnesota,58746
321,Windham,Connecticut,116782
1851,Franklin,New York,50022
...,...,...,...
211,Madera,California,157327
1293,Midland,Michigan,83156
2722,Hood,Texas,61643
1097,Pike,Kentucky,57876


In [162]:
# make a combined county, state naem for geocoding and foursquare usage
sample_list['full_loc_name']=sample_list['County or equivalent']+ ', ' + sample_list['State or equivalent'] + ', USA'
sample_list

,County or equivalent,State or equivalent,Population (2019 estimate),full_loc_name
1225,Berkshire,Massachusetts,124944,"Berkshire, Massachusetts, USA"
335,Clay,Florida,219252,"Clay, Florida, USA"
1376,Otter Tail,Minnesota,58746,"Otter Tail, Minnesota, USA"
321,Windham,Connecticut,116782,"Windham, Connecticut, USA"
1851,Franklin,New York,50022,"Franklin, New York, USA"
...,...,...,...,...
211,Madera,California,157327,"Madera, California, USA"
1293,Midland,Michigan,83156,"Midland, Michigan, USA"
2722,Hood,Texas,61643,"Hood, Texas, USA"
1097,Pike,Kentucky,57876,"Pike, Kentucky, USA"


### Step 3 Get Venues

In [166]:
c_venues=getNearbyVenues(sample_list['full_loc_name'])
c_venues.shape

Berkshire, Massachusetts, USA
Clay, Florida, USA
Otter Tail, Minnesota, USA
Windham, Connecticut, USA
Franklin, New York, USA
Calhoun, Michigan, USA
Comanche, Oklahoma, USA
Darlington, South Carolina, USA
Polk, Iowa, USA
Knox, Tennessee, USA
Wilson, North Carolina, USA
Garfield, Oklahoma, USA
Dubuque, Iowa, USA
Lea, New Mexico, USA
Catoosa, Georgia, USA
Aguadilla Municipality, Puerto Rico, USA
failed to obtain information for Aguadilla Municipality, Puerto Rico, USA
Hancock, Ohio, USA
Bedford, Virginia, USA
Troup, Georgia, USA
New London, Connecticut, USA
Cass, Michigan, USA
Lenawee, Michigan, USA
Lafayette Parish, Louisiana, USA
Newton, Georgia, USA
Navarro, Texas, USA
Halifax, North Carolina, USA
Marathon, Wisconsin, USA
Hamilton, Indiana, USA
James City, Virginia, USA
Hampshire, Massachusetts, USA
Williamson, Tennessee, USA
Crow Wing, Minnesota, USA
Clark, Ohio, USA
Mobile, Alabama, USA
Medina, Texas, USA
Tuscarawas, Ohio, USA
Howard, Indiana, USA
Catawba, North Carolina, USA
Dougla

(3044, 5)

In [167]:
c_venues.head()

,County,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Berkshire, Massachusetts, USA",Ashuwillticook Rail Trail,42.489930,-73.203533,Trail
1,"Berkshire, Massachusetts, USA",Victoria's Secret PINK,42.493085,-73.206523,Lingerie Store
2,"Berkshire, Massachusetts, USA",Bath & Body Works,42.493333,-73.206562,Cosmetics Shop
3,"Berkshire, Massachusetts, USA",Victoria's Secret,42.493105,-73.206339,Lingerie Store
4,"Berkshire, Massachusetts, USA",Starbucks,42.495168,-73.206146,Coffee Shop


#### Note
> In addtion to above data also grab the one for the home county so the comparison can be included

In [179]:
venues_list=[]
NEAR='Bonneville, Idaho, USA'
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    NEAR, 
    LIMIT)
            
    # make the GET request
try:
    home_county = requests.get(url).json()["response"]['groups'][0]['items']
except:
    print("failed to obtain information for " + name)
    
venues_list.append([(
        NEAR, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

home_county= pd.DataFrame([item for venue_list in venues_list for item in venue_list])
home_county.columns = ['County',
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']

In [182]:
home_county.head()

,County,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bonneville, Idaho, USA",Reed's Dairy Inc.,43.496945,-112.084158,Ice Cream Shop
1,"Bonneville, Idaho, USA",Starbucks,43.495026,-112.050354,Coffee Shop
2,"Bonneville, Idaho, USA",Barnes & Noble,43.480679,-111.988716,Bookstore
3,"Bonneville, Idaho, USA",Abracadabra's,43.479221,-111.991601,Breakfast Spot
4,"Bonneville, Idaho, USA","MacKenzie River Pizza, Grill & Pub",43.484666,-112.050473,American Restaurant


### Step 4 Onehot and Combine with Latitude and Longitutde Values

In [183]:
c_onehot = pd.get_dummies(c_venues[['Venue Category']], prefix="", prefix_sep="")

c_onehot['County'] = c_venues['County'] 

fixed_columns = [c_onehot.columns[-1]] + list(c_onehot.columns[:-1])
c_onehot = c_onehot[fixed_columns]

c_onehot.head()

,County,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Watch Shop,Water Park,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
0,"Berkshire, Massachusetts, USA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Berkshire, Massachusetts, USA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Berkshire, Massachusetts, USA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Berkshire, Massachusetts, USA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Berkshire, Massachusetts, USA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [185]:
home_onehot = pd.get_dummies(home_county[['Venue Category']], prefix="", prefix_sep="")

home_onehot['County'] = home_county['County'] 

fixed_columns = [home_onehot.columns[-1]] + list(home_onehot.columns[:-1])
home_onehot = home_onehot[fixed_columns]

home_onehot.head()

,County,American Restaurant,Bar,Baseball Stadium,Bookstore,Breakfast Spot,Brewery,Burger Joint,Coffee Shop,Department Store,...,History Museum,Ice Cream Shop,Mexican Restaurant,Multiplex,Park,Pizza Place,Pub,Restaurant,South American Restaurant,Steakhouse
0,"Bonneville, Idaho, USA",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,"Bonneville, Idaho, USA",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bonneville, Idaho, USA",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bonneville, Idaho, USA",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bonneville, Idaho, USA",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Note
> add the counties and the home county together so we now gauranteed have the home county with the others

In [189]:
#using append will place nan's wherever the columns don't match. In this case those need be replacd to zero
c_onehot=c_onehot.append(home_onehot, ignore_index=True).fillna(0)

In [197]:
c_onehot=c_onehot.groupby('County').mean().reset_index()

In [198]:
c_onehot

,County,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Water Park,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit,South American Restaurant
0,"Ada, Idaho, USA",0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
1,"Aiken, South Carolina, USA",0.000000,0.0,0.0,0.0,0.12,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
2,"Albany, New York, USA",0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.04,0.0,0.0,0.04,0.0,0.0,0.0
3,"Anderson, Texas, USA",0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
4,"Aroostook, Maine, USA",0.000000,0.0,0.0,0.0,0.28,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,"Winnebago, Illinois, USA",0.076923,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
145,"Winona, Minnesota, USA",0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
146,"Wood, Ohio, USA",0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0
147,"Yakima, Washington, USA",0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0


#### Note
> Here we merge the lat and long into the onehot data. To do so we will get the center of each group of veneus in an area. I am doing this by averageing the coordinates which will give alright results in small areas like within a city or so but not across a state or larger due to spherical corrections ebign ignored. A better method would be to to use a centroid algorithm or convert to cartesian vectors adn then average there and reform back into lat and long. Since those methods are very complicated and somewhat hard ot implemetn by hand and rather than geocode which gives me poor results for county center vs venue/popultion center as most county population centers are not near the physical center.

In [273]:
lat_long_temp=c_venues.append(home_county).groupby('County', as_index=False).mean()
full_data=c_onehot.join(lat_long_temp.set_index('County'), on='County', how='outer')

full_data.head(50)

,County,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit,South American Restaurant,Venue Latitude,Venue Longitude
0,"Ada, Idaho, USA",0.0,0.00,0.00,0.00,0.040000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,43.607061,-116.255929
1,"Aiken, South Carolina, USA",0.0,0.00,0.00,0.00,0.120000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,33.533609,-81.722712
2,"Albany, New York, USA",0.0,0.00,0.00,0.00,0.040000,0.000,0.00,0.00,0.00,...,0.00,0.04,0.00,0.000000,0.04,0.000000,0.0,0.00,42.671185,-73.783836
3,"Anderson, Texas, USA",0.0,0.00,0.00,0.00,0.000000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,30.572543,-96.025679
4,"Aroostook, Maine, USA",0.0,0.00,0.00,0.00,0.280000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,46.754756,-68.058131
5,"Ashland, Ohio, USA",0.0,0.00,0.00,0.00,0.000000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,40.867347,-82.300123
6,"Autauga, Alabama, USA",0.0,0.00,0.00,0.00,0.040000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,32.461920,-86.410874
7,"Baldwin, Alabama, USA",0.0,0.00,0.00,0.00,0.040000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,30.391676,-87.734956
8,"Bartow, Georgia, USA",0.0,0.00,0.00,0.00,0.000000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,32.879681,-82.474015
9,"Bedford, Virginia, USA",0.0,0.00,0.00,0.00,0.080000,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,37.337952,-79.514119


#### Note
> Home county of Bonneville is index 12 this will be used to identify cluster later on

In [274]:
full_data['Venue Latitude']

0      43.607061
1      33.533609
2      42.671185
3      30.572543
4      46.754756
         ...    
144    42.287600
145    44.043422
146    41.483784
147    46.596410
148    39.099025
Name: Venue Latitude, Length: 149, dtype: float64

### Step 5 Run K-Means
 > This will group the counties into similar groups based on the available venues

In [275]:
kclusters = 10

full_data_kmeans = full_data.drop(['County', 'Venue Latitude', 'Venue Longitude'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(full_data_kmeans)

kmeans.labels_[0:10]

array([1, 2, 1, 0, 2, 1, 2, 2, 7, 3])

#### Note
> combine cluster labels onto full_data

In [276]:
full_data.insert(0, 'Cluster Labels', kmeans.labels_)

full_data

,Cluster Labels,County,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit,South American Restaurant,Venue Latitude,Venue Longitude
0,1,"Ada, Idaho, USA",0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,43.607061,-116.255929
1,2,"Aiken, South Carolina, USA",0.000000,0.0,0.0,0.0,0.12,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,33.533609,-81.722712
2,1,"Albany, New York, USA",0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,...,0.0,0.04,0.0,0.0,0.04,0.0,0.0,0.0,42.671185,-73.783836
3,0,"Anderson, Texas, USA",0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,30.572543,-96.025679
4,2,"Aroostook, Maine, USA",0.000000,0.0,0.0,0.0,0.28,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,46.754756,-68.058131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,3,"Winnebago, Illinois, USA",0.076923,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,42.287600,-89.246197
145,1,"Winona, Minnesota, USA",0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,44.043422,-91.627173
146,1,"Wood, Ohio, USA",0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,41.483784,-83.623386
147,1,"Yakima, Washington, USA",0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,46.596410,-120.510859


## Results

### Map of all data

In [282]:
map_clusters = folium.Map(location=[39.833333333333336, -97.41666666666667], zoom_start=2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(full_data['Venue Latitude'], full_data['Venue Longitude'], full_data['County'], full_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [278]:
# find the cluster label for the home county which we saw above was index 12
cluster_number=full_data.iloc[12][0]
print(cluster_number, "is our similar palces cluster number")

2 is our simlar palces cluster number


In [283]:
bool_list=full_data['Cluster Labels']==cluster_number
similar_places_list=full_data[bool_list]
similar_places_list['County']

1                     Aiken, South Carolina, USA
4                          Aroostook, Maine, USA
6                          Autauga, Alabama, USA
7                          Baldwin, Alabama, USA
12                        Bonneville, Idaho, USA
14                            Brazos, Texas, USA
18                       Calumet, Wisconsin, USA
20                  Catawba, North Carolina, USA
21                         Catoosa, Georgia, USA
23                       Chisago, Minnesota, USA
26                            Clay, Florida, USA
27                        Clinton, Michigan, USA
29                        Columbia, Georgia, USA
32                     Crow Wing, Minnesota, USA
40     Fairbanks North Star Borough, Alaska, USA
43                       Franklin, New York, USA
45                        Genesee, New York, USA
46                   Grafton, New Hampshire, USA
47                         Guadalupe, Texas, USA
51               Hampton, City of, Virginia, USA
54                  

As is normal wiht K-means each time it is ran it comes out different. In my case I ran it with  various k's a few times over to find a good k. In the end I went with 10 for k as that gave me a result list of around 50 to 60 places out of the 150 sample I started with. I think the high k was necessary due to the large number of counties and would probably have to be higher with the full list of counties I had initially with ~850 counties. Interestingly, each time I ran k-means with the same k I got very similar results and my target group seemed to stay fairly consistent

## Discussion

From the map we can easily find the home county of Bonneville, Idaho if you know Idaho geography (it is in the southeast part of the state). Also, it turns out that there are many places that are similar. None of them are right in big cities but near them. This is good as it means that there are multiple places to live similar to my home county that will due to prosmity to biger cities likely have better jobs and wages. Interseting to note is that there was not a hidden geographical bias in the data. That means that my results in other words my target are not grouped into one area but instead are spread thoughout the US fairly well matching with population centers. Anohter thing to note is that with teh final results, it would be beneficial to do a secondary analysis of those areas with addtional demographic, housing, price, and job data. However, that is outside the scope of this analysis which was simply to find similar areas.


## Conclusion

Based on the results, I would suggest living where work can be found. As with either a reasonable commute or some effort in the job hunting, one can find a place like home in more places than they would initially think